In [1]:
from scara import *
import spline
from pylab import *
#%pylab inline

In [2]:
def kinematics_test_plot(pos, a, b, limits):
    import numpy as np
    x = pos[0]
    y = pos[1]
    (alpha, beta) = inverse_kinematics((x, y), a, b, limits)
    print(alpha, beta)
    (x_f, y_f) = forward_kinematics((alpha, beta), a, b)
    plot([0, a*np.cos(alpha), x_f], [0, a*np.sin(alpha), y_f])

In [3]:
a = 140
b = 52
limits = [[-3.14/180*120, 3.14/180*120], [-3.14/180*125, 3.14/180*125]]

points = [[192, 0], [140, 52], [100, 100], [100, 150], [0, 192]]

points_inv = [inverse_kinematics(p, a, b, limits) for p in points]
points_check = [forward_kinematics(p, a, b) for p in points_inv]
points_check

[(192.0, 0.0),
 (140.00000000000003, 51.999999999999964),
 (100.00000000000003, 100.0),
 (100.00000000000003, 150.0),
 (1.1756609271814592e-14, 192.0)]

In [4]:
traj = spline.SplineTrajectory(points_inv, start_dir=None, end_dir=None, roundness=0.8)
resolution = 0.1
sampled_traj = traj.get_sample_points(resolution)
positions = [point.position for point in sampled_traj]
positions = [forward_kinematics(p, a, b) for p in positions]
scatter(*zip(*positions))
xlabel('x')
ylabel('y')

for p in points:
    hold(True)
    kinematics_test_plot(p, a, b, limits)
    
grid(True)
axis('equal')
show()

0.0 0.0
-2.77555756156e-16 1.57079632679
0.413845368118 1.72970604707
0.775397535102 0.795014065288
1.57079632679 0.0


In [5]:
import trajectory

def actuator_limits(position, velocity):
    return ([-0.5, -0.5], [0.5, 0.5])

tp = trajectory.compute_trajectory(traj, actuator_limits, 1, 0.05, v_limit=20)

In [6]:
dyn_pts = list(zip(*tp))
pos_t = list(zip(*dyn_pts[0]))
vel_t = list(zip(*dyn_pts[1]))
acc_t = list(zip(*dyn_pts[2]))

quiver(pos_t[0], pos_t[1], np.array(vel_t[0]), np.array(vel_t[1]), width=0.0022, pivot='tail', color='r')
quiver(pos_t[0], pos_t[1], np.array(acc_t[0]), np.array(acc_t[1]), width=0.0022, pivot='tail', color='g')
scatter(*pos_t, s=3)

axis('equal')
legend(['vel', 'acc'])

figure()
step(range(len(acc_t[0])), acc_t[0][:])
step(range(len(acc_t[1])), acc_t[1][:])
plot(vel_t[0])
plot(vel_t[1])
plot([np.linalg.norm(i) for i in zip(*vel_t)])
plot(pos_t[1][:])
legend(['acc 1', 'acc 2', 'vel 1', 'vel 2', '|vel|', 'pos'])
grid(True)
show()